<a href="https://colab.research.google.com/github/coderlohith12/tomato_leaf_preduction/blob/main/Sony_of_Tomato_inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf  # this is import liobraries
# import keras
# https://keras.io/api/applications/

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3  import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
# from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

# from tensorflow.keras.applications.vgg19  import VGG19

# /content/drive/MyDrive/Tomato/dataset/train

In [ ]:

tf.__version__

'2.12.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
IMAGE_SIZE = [224, 224]  ## conerting image into 224,224

train_path = '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'  # path of train data
valid_path = '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'  ## path test data

In [ ]:
inceptionv3 = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)  ## [224,224,3]  # rgb ==> red,green,bule

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
# don't train on the inception weights
for layer in inceptionv3.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/*')  # train and test

In [ ]:
folders  ## /content/drive/MyDrive/Tomato/dataset/train/Tomato___Bacterial_spot

['/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Tomato___Target_Spot',
 '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_mosaic_virus',
 '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Tomato___Septoria_leaf_spot',
 '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Tomato___Leaf_Mold',
 '/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Disease

In [ ]:
x = Flatten()(inceptionv3.output)  ## changimg the array size to 1-d array   # multi dim -- > 1d

In [ ]:
len(folders)

10

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)  # sigmoid 0,1  ## 10 desies
## fully connected layer
# create a model object
model = Model(inputs=inceptionv3.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model.compile(
  loss = 'categorical_crossentropy',  ## output categorical
  optimizer = "adam",  ## recude our loss score ["adam,sgd,gd,msgd"]
  metrics = ['accuracy']  ## accuary
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, #  [0,1]
                                   shear_range = 0.2,  ## data arugmention
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,  ## key roles ## 64
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/BBA project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 18346 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


In [ ]:
18346 /32

573.3125

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  # verbose=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)



## bachpropagation adn front propogation

<ipython-input-18-9f0c6c21ce70>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
283/574 [=============>................] - ETA: 1:43:22 - loss: 3.5357 - accuracy: 0.6305

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.title("loss_values_figure")
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.title("Accuracy_values_figure")
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_inception.h5') ## h5 ## all your model will be storing in this h5 file ## this file is given to fornt end to execute the application


# model.summary()

In [ ]:
# templates
# colab notenobk
# model link --->
# https://keras.io/api/applications/
# https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
!nvidia-smi